In [1]:
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import toolkit.filter

import tensorflow as tf
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from optuna.integration import KerasPruningCallback


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for k in range(len(physical_devices)):
        tf.config.experimental.set_memory_growth(physical_devices[k], True)
        print('memory growth:', tf.config.experimental.get_memory_growth(physical_devices[k]))
else:
    print("Not enough GPU hardware devices available")

Not enough GPU hardware devices available


In [3]:
class DatasetClass:
    def __init__(self, data_path):
        self.path = data_path
    
    def __reflect_index(self, data, index):
        if index != None:
            data = data[:, index]
        return data
        
    def __load_df(self, data_label):
        data_x = pd.read_csv(f"{self.path}/{data_label}_nx.csv", index_col=0)
        data_y = pd.read_csv(f"{self.path}/{data_label}_ny.csv", index_col=0)
        return data_x, data_y
    
    def __load_data(self, data_label, x_index, y_index):
        data_x, data_y = self.__load_df(data_label)
        data_x = self.__reflect_index(data_x.values, x_index)
        data_y = self.__reflect_index(data_y.values, y_index)
        return data_x, data_y
    
    def __load_stack(self, dataset_list, x_index, y_index):
        for label in dataset_list:
            tmp_x, tmp_y = self.__load_data(label, x_index, y_index)
            if dataset_list.index(label) == 0:
                data_x = tmp_x
                data_y = tmp_y
            else:
                data_x = np.vstack((data_x, tmp_x))
                data_y = np.vstack((data_y, tmp_y))
        return data_x, data_y
    
    def __load_dict(self, dataset_list, x_index, y_index):
        data_x, data_y = {}, {}
        for label in dataset_list:
            tmp_x, tmp_y = self.__load_data(label, x_index, y_index)
            data_x[label] = tmp_x
            data_y[label] = tmp_y
        return data_x, data_y
    
    def get_data(self, dataset_label, x_index=None, y_index=None, dict_type:bool=False):
        if not dict_type:
            if type(dataset_label) == str:
                data_x, data_y = self.__load_data(dataset_label, x_index, y_index)
            else:
                data_x, data_y = self.__load_stack(dataset_label, x_index, y_index)
        else:
            data_x, data_y = self.__load_dict(dataset_label, x_index, y_index)
        return data_x, data_y
    
    def get_dataframe(self, dataset_list):
        data_x = {}
        data_y = {}
        for label in dataset_list:
            tmp_x, tmp_y = self.__load_df(label)
            data_x[label] = tmp_x
            data_y[label] = tmp_y
        return data_x, data_y


def set_index(model_type):
    if "conv." in model_type:
        x_index = [i for i in range(7)]
    elif model_type == "prop.1":
        x_index = [0,3,4,6]
    elif model_type == "prop.2":
        x_index = [1,2,5,6]
    elif model_type == "prop.3":
        x_index = [1,2,5,6]
    elif model_type == "prop.4":
        x_index = [0,3,4,5,6]
    else:
        print(f"<< {model_type} >> This model is not exist.")
    y_index = [int(model_type[-1])-1]
    return x_index, y_index


def set_dataset_label(model_type):
    type_id = int(model_type[-1])
    if type_id == 1:
        learn = "ms1a"
        test = "ms2a"
    elif type_id == 2:
        learn = "ms3a"
        test = "ms1a"
    elif type_id == 3:
        learn = "ms2a"
        test = "ms3a"
    elif type_id == 4:
        learn = "ms3a"
        test = "ms1a"
    return learn, test


In [4]:
model_tag = "prop.4"
data_path = f"./dataset/norms"

x_index, y_index = set_index(model_tag)
learn_list, test_list  = set_dataset_label(model_tag)

dataset = DatasetClass(data_path)
learn_x, learn_y = dataset.get_data(learn_list, x_index, y_index)
test_x, test_y = dataset.get_data(test_list, x_index, y_index)

print(f"x_index: {x_index}, y_index: [{y_index[0]}]")
print(f"learn list: {learn_list}, test_list: {test_list}")


x_index: [0, 3, 4, 5, 6], y_index: [3]
learn list: ms3a, test_list: ms1a


In [5]:
study_label = "ver1.0"
STUDY_LOADING = False

storage_path = f"sqlite:///optimize_{model_tag}.db"
study_name = model_tag + "_" + study_label

# study load or create
if STUDY_LOADING:
    study = optuna.load_study(study_name, storage_path, pruner=optuna.pruners.MedianPruner())
else:
    study = optuna.create_study(study_name=study_name, storage=storage_path, direction="minimize", pruner=optuna.pruners.MedianPruner())


[I 2020-07-21 03:22:53,658] A new study created with name: prop.4_ver1.0


In [6]:
batch_size = 1024
element = 3592

epochs = 200
samples = 51
input_unit = learn_x.shape[1]
testdata_size = test_x.shape[0]

filter = toolkit.filter.Filter_with_IQR()

def objective(trial):
    max_unit1 = 100
    max_unit2 = 100
    
    num_unit1 = trial.suggest_int(f"num_unit1", 1, max_unit1)
    buf_unit2 = int((element - input_unit * num_unit1)/(num_unit1 + 1))
    if max_unit2 >= buf_unit2:
        max_unit2 = buf_unit2
    num_unit2 = trial.suggest_int(f"num_unit2", 1, max_unit2)
    
    score_list = [np.nan for i in range(samples)]
    for i in range(samples):
        clear_session()
        model = Sequential([
            Dense(input_dim=input_unit, units=num_unit1, activation="tanh", kernel_initializer="glorot_uniform"),
            Dense(input_dim=num_unit1, units=num_unit2, activation="tanh", kernel_initializer="glorot_uniform"),
            Dense(input_dim=num_unit2, units=1, kernel_initializer="glorot_uniform")
        ])
        model.compile(loss="mse", optimizer=Adam())
        model.fit(learn_x, learn_y, batch_size=batch_size, epochs=epochs, verbose=0)
        score = model.evaluate(test_x, test_y, batch_size=testdata_size, verbose=0)
        score_list[i] = score
        print(f"\r#{trial.number:2}: units={[num_unit1, num_unit2]}, sample {i}/{samples}, score:{score:.4e}", end="")
        
    df_score = filter.filtering(score_list)
    count, mean, std = df_score.describe().loc[["count","mean","std"]]
    print(f"\r#{trial.number:2}: units={[num_unit1, num_unit2]}: samples={int(count)}/{samples}, mean={mean:.4e}, std={std:.4e}\n")
    return mean

In [ ]:
study.optimize(objective, n_trials=100)

print("\n")
print("*** All Trial are finished!! ***")

# 0: units=[89, 15]: samples=46/51, mean=2.0298e-06, std=9.2221e-07



[I 2020-07-21 03:29:01,094] Finished trial#0 with value: 2.0298496635810492e-06 with parameters: {'num_unit1': 89, 'num_unit2': 15}. Best is trial#0 with value: 2.0298496635810492e-06.


# 1: units=[80, 25]: samples=46/51, mean=1.8964e-06, std=6.8224e-07



[I 2020-07-21 03:35:07,322] Finished trial#1 with value: 1.8964499885066248e-06 with parameters: {'num_unit1': 80, 'num_unit2': 25}. Best is trial#1 with value: 1.8964499885066248e-06.


# 2: units=[52, 43]: samples=45/51, mean=2.1196e-06, std=1.1069e-06



...

#13: units=[73, 30]: samples=46/51, mean=3.0271e-06, std=1.7476e-06



[I 2020-07-21 04:46:11,909] Finished trial#13 with value: 3.027107477069164e-06 with parameters: {'num_unit1': 73, 'num_unit2': 30}. Best is trial#1 with value: 1.8964499885066248e-06.


#14: units=[77, 28]: samples=43/51, mean=1.9086e-06, std=9.0680e-07



[I 2020-07-21 04:52:43,115] Finished trial#14 with value: 1.908601477627472e-06 with parameters: {'num_unit1': 77, 'num_unit2': 28}. Best is trial#1 with value: 1.8964499885066248e-06.


#15: units=[60, 39], sample 13/51, score:1.1915e-06